<a href="https://colab.research.google.com/github/narfdf999/Arquitectura/blob/master/producto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Solución final al problema de generar edificio a lo largo de un camino***




# Introducción

---

En los últimos meses se ha avanzado mucho en lo correspondiente a generación de plantas de edificios y su distribución. El enfoque el éste trabajo es radicalmente distinto


#Librerías

---



In [0]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model ,Sequential, load_model
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

# Visualización y preprocesamiento de datos

---



# Implementación del sistema

---



### Optimizador ADAM

In [0]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

## GAN generadora de la primera visual

---



### Generador





In [0]:
# Red deconvolucional cuyo input es un vector de ruido de 128 posiciones con opción a condicionar que genera como output una matriz de ( , ) donde se almacenan las distancias del punto de la curva 
# al edificio en una proyección de Mercator. 

### Discriminador

### GAN

## Red neuronal que avanza paso a paso por el camino y genera las visuales según avanza

---



# Entrenamiento

---



# Postprocesado de datos de salida

---

